In [1]:
#
%run ./connectors.ipynb
%run ./my_utils.ipynb

/var/folders/52/16l8f4r11n1b1bthpmm5505w0000gn/T/ipykernel_33068/1435207182.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=127259169819-6jg2fe7gb3mgpj4kog72t599c26uekat.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54289%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&state=huBX9Y3SQkEb0ZkSlosPcHPkVQ1BYY&access_type=offline


<>:6: SyntaxWarning: invalid escape sequence '\.'
/var/folders/52/16l8f4r11n1b1bthpmm5505w0000gn/T/ipykernel_33068/1242829460.py:6: SyntaxWarning: invalid escape sequence '\.'
  df[column].str.replace("\..*", "", regex=True) if df[column].dtype == 'object' else df[column],


In [459]:
#upload model pickle
with open('xgb_model_v1_model.pkl', 'rb') as file:
    xgb_model_loaded = pickle.load(file)

In [ ]:
dataset = load_sql('dataset.sql')
df= client.query(dataset).to_dataframe()

In [ ]:
with open('debugging_X_var.txt', 'r') as f:
    column_list = [line.strip() for line in f.readlines()]

In [ ]:
# Convert all string values in the DataFrame to lower case
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [438]:
## define hardcoded_features
top_email_domain = ['gmail.com', 'yahoo.com', 'privaterelay.appleid.com', 'icloud.com']
top_channel = ['google', 'app', 'facebook', 'invite friend', 'sidelines'] 
geo_loc = ['california', 'texas', 'florida', 'illinois', 'georgia']
platform = ['web', 'app']


#####features apply dummy variables function
df = process_column(df, "signup_email_domain", top_email_domain)
df = process_column(df, "channel", top_channel)
df = process_column(df, "first_geo_region", geo_loc)
df = process_column(df, "signup_platform", platform)

In [439]:
##transform the columns into log and drop

columns_to_transform = [
    'total_bet_amt',
    'total_win_amt_sc',
    'total_win_amt_cc',
    'sum_redeem_amt',
    'sum_amt_total',
    'sum_sc_coin_total'
]

df = log_transform_columns(df, columns_to_transform)

In [440]:
df['time_diff_minutes'] = (df['first_purchase_date'] - df['signup_date']).dt.total_seconds() / 3600

In [447]:
df = df.fillna(0)

In [449]:
missing_columns = [col for col in column_list if col not in df.columns]

if missing_columns:
    print(f"Error: Missing columns: {', '.join(missing_columns)}")
    exit(1)  # Exit if any column is missing

# Generate new variable X from df including the required columns
X = df[column_list]

In [453]:
probabilities = xgb_model_loaded.predict_proba(X)
df['probability_positive_class'] = probabilities[:, 1]

df['potential_vip_ind'] = 0

# Filter users with probability above 0.9
above_threshold = df[df['probability_positive_class'] > 0.9]

# Randomly select half of those to receive a potential VIP indicator
num_to_mark = len(above_threshold) // 2
if num_to_mark > 0:
    selected_indices = above_threshold.sample(num_to_mark).index
    df.loc[selected_indices, 'potential_vip_ind'] = 1

# Create a new DataFrame with adjusted probabilities
df_vip = df[df['probability_positive_class'] > 0.9].copy()
df_vip.loc[~df_vip.index.isin(selected_indices), 'probability_positive_class'] = np.nan

ValueError: feature_names mismatch: ['avg_spin_amt', 'rtp_ratio', 'ggr_amt', 'logins_first_96h', 'total_bet_txn', 'max_purchase_amt_horly', 'max_redeem_amt_hourly', 'sum_redeem_txn', 'cnt_txn_total', 'sum_spin_sc', 'sum_spin_cc', 'max_eod_sc_balance', 'sum_bonus_wheel', 'sum_bonus_plinko', 'sum_offer_amt', 'sum_bonus_amt', 'sum_daily_mission', 'sum_leaderboard_reward', 'time_diff_minutes', 'days_of_activity', 'max_bet_amt', 'min_bet_amount', 'max_win_amt', 'avg_rounds_bet', 'avg_bet_amt', 'max_daily_games_unique', 'sum_bonus_promotion_bet_amount', 'signup_email_domain_gmail', 'signup_email_domain_icloud', 'signup_email_domain_other', 'signup_email_domain_privaterelay', 'signup_email_domain_yahoo', 'channel_App', 'channel_Invite Friend', 'channel_facebook', 'channel_google', 'channel_other', 'first_geo_region_other', 'signup_platform_app', 'ln_total_bet_amt', 'ln_total_win_amt_sc', 'ln_total_win_amt_cc', 'ln_sum_redeem_amt', 'ln_sum_amt_total', 'ln_sum_sc_coin_total', 'first_purchase_am

In [136]:
# Save DataFrame to a CSV file
df_vip.to_csv('df_vip_sep29.csv', index=False)